In [ ]:
import socket

afqdn = socket.getfqdn() # your /etc/hosts should be set up to contain fully qualified domain name of the machine
afqdn

## 1. Make sure you can run the `fx` command.

## 2. Create a workspace for the new federation project.

In [ ]:
!fx workspace create --prefix ~/workspace --template torch_cnn_histology
%cd ~/workspace

## 3. Initialize the plan
Although it is possible to train models from scratch, it is assumed that in many cases the federation may perform fine-tuning of a previously-trained model. For this reason, the pre-trained weights for the model will be stored within protobuf files on the aggregator and passed to the collaborators during initialization. As seen in the YAML file, the protobuf file with the initial weights is expected to be found in the file `torch_cnn_mnist_init.pbuf`. Let's just create an initial set of random model weights and put it into that file by running the command:

In [ ]:
!fx plan initialize -a $afqdn

## 4. Run the Certificate Authority command.
This will setup the Aggregator node as the Certificate Authority for the Federation. All certificates will be signed by the aggregator. The command will create a simple database file to keep track of all issued certificates.

In [ ]:
!fx workspace certify

## 5. Run the aggregator certificate creation command.

In [ ]:
!fx aggregator generate-cert-request --fqdn $afqdn

## 6. Run the aggregator certificate signing command

In [ ]:
!fx aggregator certify -s --fqdn $afqdn

## 7. Export the workspace so that it can be imported to the collaborator nodes.

In [ ]:
!fx workspace export

## 14. Sign collaborators certificates

In [ ]:
!fx collaborator certify -s --request-pkg col_one_to_agg_cert_request.zip

In [ ]:
!fx collaborator certify -s --request-pkg col_two_to_agg_cert_request.zip

## Start the aggregator.

In [ ]:
!fx aggregator start

At this point, the aggregator is running and waiting for the collaborators to connect. When all of the collaborators connect, the aggregator starts training. When the last round of training is complete, the aggregator stores the final weights